# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [1]:
# your code here
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
import tensorflow.keras as keras
import tensorflow as tf

# 1. Read `tic-tac-toe.csv` into a dataframe.
tictactoe_array = pd.read_csv('tic-tac-toe.csv')

# 1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
tictactoe_array.head() # there's an x, o, and empty(b)

TL TM TR ML MM MR BL BM BR  class
0  x  x  x  x  o  o  x  o  o   True
1  x  x  x  x  o  o  o  x  o   True
2  x  x  x  x  o  o  o  o  x   True
3  x  x  x  x  o  o  o  b  b   True
4  x  x  x  x  o  o  b  o  b   True

In [2]:
# 1. Convert the categorical values to numeric in all columns.
tictactoe_array.dtypes

TL       object
TM       object
TR       object
ML       object
MM       object
MR       object
BL       object
BM       object
BR       object
class      bool
dtype: object

##### Separate the inputs and output.


In [3]:
target = tictactoe_array['class']

In [4]:
features = tictactoe_array[[col for col in tictactoe_array.columns if col != 'class']]

In [5]:
features.head()

TL TM TR ML MM MR BL BM BR
0  x  x  x  x  o  o  x  o  o
1  x  x  x  x  o  o  o  x  o
2  x  x  x  x  o  o  o  o  x
3  x  x  x  x  o  o  o  b  b
4  x  x  x  x  o  o  b  o  b

In [6]:
for col in features.columns:
    features[col] = np.where(
        features[col] == 'x',
        1,
        np.where(
            features[col] == 'o',
            -1, 
            0))


features.head()

/Users/sampoad/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


TL  TM  TR  ML  MM  MR  BL  BM  BR
0   1   1   1   1  -1  -1   1  -1  -1
1   1   1   1   1  -1  -1  -1   1  -1
2   1   1   1   1  -1  -1  -1  -1   1
3   1   1   1   1  -1  -1  -1   0   0
4   1   1   1   1  -1  -1   0  -1   0

In [7]:
features.shape # so there's 958 rows. 

(958, 9)

In [8]:
958 * 0.1

95.80000000000001

In [9]:
958-96

862

so take the first 90% of the rows, 0-862

In [10]:
train_X = features[:862]
train_y = target[:862]

test_X = features[862:]
test_y = target[862:]

In [11]:
train_X.head()

TL  TM  TR  ML  MM  MR  BL  BM  BR
0   1   1   1   1  -1  -1   1  -1  -1
1   1   1   1   1  -1  -1  -1   1  -1
2   1   1   1   1  -1  -1  -1  -1   1
3   1   1   1   1  -1  -1  -1   0   0
4   1   1   1   1  -1  -1   0  -1   0

In [12]:
train_X.shape

(862, 9)

In [13]:
# REFACTOR, because the above gave very low accuracy (32%)

from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(
	features,
	target,
	test_size  = 0.2,
	random_state = 42,
)



In [14]:
# 1. Normalize the input data.
# Built in Normalizer in Keras
train_X = tf.keras.utils.normalize(train_X, axis = 1)
test_X = tf.keras.utils.normalize(test_X, axis = 1)


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [15]:
# your code here
# already split the training and test data, and scaled that

model = tf.keras.models.Sequential() # create a Sequential model

model.add(tf.keras.layers.Flatten()) # add our first layer

model.add(tf.keras.layers.Dense( # add a hidden layer
    128, # number of nodes
    activation = tf.nn.relu, # use the ReLU function https://www.tensorflow.org/api_docs/python/tf/nn/relu
))

model.add(tf.keras.layers.Dense(128, activation = tf.nn.relu)) # add another hidden layer

model.add(tf.keras.layers.Dense( # add our final layer
    2, # binary, either Player X Wins or there's a loss (O wins, draw)
    activation = tf.nn.softmax, # using a different function here at the end
))

model.compile( # if you don't know why to change something, leave it
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy'], # report this specific metric of the accuracy
)

In [16]:
train_X.shape

(766, 9)

In [17]:
train_y.shape

(766,)

In [18]:
# fit the data to the model

model.fit(
    train_X.values, # .values.reshape(-1, 1).ravel(), # this creates too large a list
    train_y.values,
    epochs = 3,
) # epochs is the number of passes through the data

Train on 766 samples
Epoch 1/3
766/766 [==============================] - 0s 287us/sample - loss: 0.6058 - accuracy: 0.7063
Epoch 2/3
766/766 [==============================] - 0s 31us/sample - loss: 0.5196 - accuracy: 0.7493
Epoch 3/3
766/766 [==============================] - 0s 34us/sample - loss: 0.4628 - accuracy: 0.7820


In [19]:
# Evaluate the model

val_loss, val_acc = model.evaluate(test_X, test_y) # 

print(f'Loss: {val_loss}')
print(f'Accuracy: {val_acc}')

192/192 [==============================] - 0s 319us/sample - loss: 0.4413 - accuracy: 0.7969
Loss: 0.44127003848552704
Accuracy: 0.796875


In [20]:
# Save the model
model.save('tic-tac-toe.model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: tic-tac-toe.model/assets


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [21]:
# your code here
ttt_model = tf.keras.models.load_model('tic-tac-toe.model')

In [23]:
predictions = ttt_model.predict(test_X.values)

predictions[0] # returns array([0.19566773, 0.80433226] which means the model is 80% sure that the X player does NOT win on the first pull from test_X

array([0.19566773, 0.80433226], dtype=float32)

In [27]:
# so check and see what test_y is
test_y.values[0] # returns FALSE, which we know to mean that PlayerX does not win, so my prediction was accurate on this case!

False

In [28]:
predictions[5]

array([0.04557356, 0.95442647], dtype=float32)

In [30]:
test_y.values[5] # in this case, however, my model was WRONG, it said a 95% probability of False, but this was true

True

In [31]:
predictions[:10]

array([[0.19566773, 0.80433226],
       [0.30058092, 0.699419  ],
       [0.4858532 , 0.51414686],
       [0.3266046 , 0.6733954 ],
       [0.8322517 , 0.16774827],
       [0.04557356, 0.95442647],
       [0.09776353, 0.9022365 ],
       [0.601863  , 0.39813694],
       [0.39516   , 0.60484   ],
       [0.33483183, 0.6651682 ]], dtype=float32)

In [33]:
test_y.values[:10]

array([False,  True,  True, False, False,  True,  True,  True,  True,
       False])

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [ ]:
# your code here

# let's try adding more layers to the model



In [ ]:
# let's adjust the learning rate

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here